In [73]:
import os
import sys
import shutil
import pandas
import numpy
import ipystata
import matplotlib.pyplot as plt

from ipystata.config import config_stata
config_stata('/home/kmongeon/research/stata14/stata-mp',force_batch=True)

from ipystata.config import config_syntax_higlight    
config_syntax_higlight(True)  

pandas.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.float_format = '{:.2f}'.format

In [74]:
pwd

'/home/kmongeon/research/exercise_physiology/scripts'

In [75]:
try: 
    shutil.rmtree('out')
    os.mkdir('out')
except:
    pass

In [76]:
%%stata --output dm

import delimited ./data/IzzyLongitudinalWorkingMarch2015.csv,  case(lower) clear 
order id session sequence
sort id sequence
save ./out/di, replace

import delimited ./data/torque.csv, case(lower) clear
drop if session==0
drop sequence
rename new_sequence sequence
order id session sequence
sort id  sequence
foreach var in ptiso pt60 pt240 {
replace `var' = . if `var'==-9999
}
save ./out/dt, replace

** Identify potentially data with errors
* Nota: July  2016
import delimited ./data/NegativeIDs.csv, case(lower) clear
keep nota id sequence 
keep if nota==1
save ./out/db1, replace

* Baraket: Nov 2016
import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
keep falk id sequence
keep if falk==1
save ./out/db2, replace

** additional ptiso data
import delimited ./data/Spring_2013_knee_extension.csv, case(lower) clear
save ./out/dk, replace


* merge
use ./out/di, clear
sort id sequence
merge 1:1 id sequence using ./out/dt,  nogen
merge 1:1 id sequence using ./out/db1, nogen
merge 1:1 id sequence using ./out/db2, nogen  
merge 1:1 id sequence using ./out/dk 
drop if _merge==2
drop _merge
count


(70 vars, 427 obs)
(note: file ./out/di.dta not found)
file ./out/di.dta saved
(14 vars, 421 obs)
(40 observations deleted)(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(note: file ./out/dt.dta not found)
file ./out/dt.dta saved
. * Nota: July  2016(32 vars, 109 obs)
(59 observations deleted)
(note: file ./out/db1.dta not found)
file ./out/db1.dta saved
. import delimited ./data/Spring_2013_knee_extension.csv, case(lower) clear
(4 vars, 50 obs)
(note: file ./out/dk.dta not found)
file ./out/dk.dta saved
. import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
(34 vars, 473 obs)
(460 observations deleted)
(note: file ./out/db2.dta not found)
file ./out/db2.dta saved
. use ./out/di, clear

    Result                           # of obs.
    -----------------------------------------
    not matched                            58
        from master                        52  
        from using           

In [77]:
%%stata --data dm --output dm

rename radius_sos rsos
rename tibial_sos tsos

rename grip_strength_best grip 
rename mat_offset_new matu

rename godin_pa godin
rename ntxcreat ntxc
rename caloricintake cint
renam calcium calc 
rename vitd vitd 
rename v48 vitd2
rename paq_total_score paq
rename totmvh mvh

rename forearm_length flength
rename thigh_length   tlength

global U id session sequence 
global Y rsos tsos
global X grip ptiso pt60 godin paq ntxc cint calc vitd vitd2 mvh
global Z matu age gender
global D nota falk

*keep $U $Y $X $Z $D

***grip only

*drop if missing(rsos)
*drop if missing(tsos)
*drop if missing(grip)
*drop if missing(ptiso) 
drop if matu>5
drop if id==517 & sequence==7

sort id sequence
by id: egen seq = seq()
global U id session sequence seq

order $U $D $Y $X $Z 
sum *
xtset id seq
count 

capture drop griph 
reg grip rsos 
predict griph 
replace griph = grip if (!missing(grip) & nota!=1)

reg ptiso matu

capture drop ptisoh  
*replace ptiso =. if ptiso>250
reg ptiso tsos
predict ptisoh 
replace ptisoh = ptiso if !missing(ptiso) 

capture drop ptisoh2  
reg ptisoh matu
predict ptisoh2 
replace ptisoh = ptisoh2 if missing(ptisoh) 
drop ptisoh2

capture drop rsosh  
reg rsos griph  
predict rsosh 
replace rsosh = rsos if (!missing(rsos) & nota!=1)

capture drop tsosh  
reg tsos ptisoh  
predict tsosh 
replace tsosh = tsos if (falk!=1 & !missing(tsos))

capture drop flengthh  
reg flength rsosh
predict flengthh 
replace flengthh = flength if !missing(flength)

capture drop tlengthh  
reg tlength tsosh
predict tlengthh 
replace tlengthh = tlength if !missing(tlength)


capture drop mvhh  
reg mvh ptisoh 
predict mvhh 
replace mvhh = mvh if !missing(mvh)

capture drop sedh  
reg totsedetary ptisoh 
predict sedh 
replace sedh = totsedetary if !missing(mvh)

capture drop ntxch  
reg ntxc rsosh 
predict ntxh 
replace ntxh = ntxc if !missing(ntxc)

capture drop godinh  
reg godin tsosh
predict godinh 
replace godinh = godin if !missing(godin)

capture drop paqh  
reg paq matu 
predict paqh 
replace paqh = paq if !missing(paq)

capture drop paqprvnth  
reg paqprvnt matu 
predict paqprvnth 
replace paqprvnth = paqprvnt if !missing(paqprvnt)

capture drop calch  
reg calc matu
predict calch 
replace calch = calc if !missing(calc)

capture drop cinth  
reg cint matu
predict cinth 
replace cinth = cint if !missing(cint)

capture drop vitdh  
reg vitd matu
predict vitdh 
replace vitdh = vitd if !missing(vitd)

sort id seq
by id: egen trips = max(seq)
order trips, after(seq)

gen boy = 1 if gender==0
gen girl = 1 if gender==1




  (all newnames==oldnames)
. ***grip only. *drop if missing(tsos). *drop if missing(ptiso) (7 observations deleted)
(1 observation deleted)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
          id |        425    326.1976    197.5498        100        583
     session |        425    1.430588    .4957421          1          2
    sequence |        425    4.197647    1.833518          1          7
         seq |        425    1.868235    .8776861          1          4
        nota |         50           1           0          1          1
-------------+---------------------------------------------------------
        falk |         11           1           0          1          1
        rsos |        415    3815.552    100.9095       3542       4140
        tsos |        415     3687.08    109.0384       3367       4015
        grip |        379    23.43807    7.364498          9      

In [78]:
dm['rsoss'] = (dm['rsos'] - dm['rsos'].mean()) / dm['rsos'].std()
dm['tsoss'] = (dm['tsos'] - dm['tsos'].mean()) / dm['tsos'].std()
dm['grips'] = (dm['grip'] - dm['grip'].mean()) / dm['griph'].std()
dm['ptisos'] = (dm['ptisoh'] - dm['ptisoh'].mean()) / dm['ptiso'].std()

dm['ntxs'] = (dm['ntx'] - dm['ntx'].mean()) / dm['ntx'].std()
dm['mvhs'] = (dm['mvh'] - dm['mvh'].mean()) / dm['mvh'].std()
dm['gods'] = (dm['godin'] - dm['godin'].mean()) / dm['godin'].std()

dm['cints'] = (dm['cint'] - dm['cint'].mean()) / dm['cint'].std()
dm['matus'] = (dm['matu'] - dm['matu'].mean()) / dm['matu'].std()

dm.head(2)

,id,session,sequence,seq,trips,nota,falk,rsos,tsos,grip,ptiso,pt60,godin,paq,ntxc,cint,calc,vitd,vitd2,mvh,matu,age,gender,height,seatedheight,leglength,weight,bf_slaughter,bia_pbf,bmi,lbmkg,yearsfromaphv,age_at_phv,bone_age,tanner,menarche,age_first_period,period_frequency,medications,vitamins,fracture,radius_z,tibial_z,wrist_flexion_best,flength,forearm_lean_csa,forearm_us_csa,anterior_csa,anteriorradius,ant_rad_csa,...,totlight,totmod,tothard,totvhard,totaltime,paqprvnt,training_data,upperarm_length,arm_circumference,arm_lean_csa,tlength,thigh_circumference,thigh_lean_csa,bicepsbrachii,v68,quads,v70,biodex_id,visit_one,visit_two,dob,agedays,ageyears,pt240,griph,ptisoh,rsosh,tsosh,flengthh,tlengthh,mvhh,sedh,ntxh,godinh,paqh,paqprvnth,calch,cinth,vitdh,boy,girl,rsoss,tsoss,grips,ptisos,ntxs,mvhs,gods,cints,matus
0,100,1,1,1,4.00,nan,nan,3828.00,3601.00,nan,113.93,69.29,37.00,2.59,711.85,1572.00,1077.67,3.38,135.27,105.71,-1.67,11.75,0,152.00,75.20,76.80,44.20,22.81,21.90,19.13,34.12,-2.13,13.88,11.16,2.00,nan,,,0.00,0.00,0.00,0.80,-0.75,nan,nan,nan,nan,nan,nan,nan,...,174.29,94.57,10.14,1.00,804.79,1.00,0.00,27.40,24.00,30.77,35.50,41.70,93.97,26.49,5.51,27.25,5.83,B6M100,05-05-2010,05-12-2010,08-11-1998,4292.00,11.75,41.80,23.77,113.93,3828.00,3601.00,23.77,35.50,105.71,78.43,711.85,37.00,2.59,1.00,1077.67,1572.00,3.38,1.00,nan,0.12,-0.79,nan,-0.22,-1.10,-0.01,-0.90,-0.05,-0.40
1,100,1,3,2,4.00,nan,nan,3898.00,3629.00,27.00,136.02,87.78,48.00,3.17,760.09,2219.01,1212.82,1.02,41.02,93.62,-0.71,12.71,0,160.80,79.60,81.20,48.80,17.24,14.90,18.87,40.39,-1.21,13.92,12.41,4.00,nan,,,0.00,0.00,0.00,1.00,-0.70,13.65,25.60,30.41,14.49,11.97,34.74,9.48,...,148.71,48.79,32.00,12.83,796.04,1.00,1.00,30.00,24.20,36.14,36.50,42.30,106.18,23.42,4.31,28.27,6.28,B6M200,04-20-2011,04-27-2011,08-11-1998,4642.00,12.71,52.60,27.00,136.02,3898.00,3629.00,25.60,36.50,93.62,75.55,760.09,48.00,3.17,1.00,1212.82,2219.01,1.02,1.00,nan,0.82,-0.53,0.53,0.19,-0.55,-0.30,-0.62,1.29,0.09


In [79]:
dm['rsoshs'] = (dm['rsosh'] - dm['rsosh'].mean()) / dm['rsosh'].std()
dm['tsoshs'] = (dm['tsosh'] - dm['tsosh'].mean()) / dm['tsosh'].std()
dm['griphs'] = (dm['griph'] - dm['griph'].mean()) / dm['griph'].std()
dm['ptisohs'] = (dm['ptisoh'] - dm['ptisoh'].mean()) / dm['ptisoh'].std()

dm['ntxhs'] = (dm['ntxh'] - dm['ntxh'].mean()) / dm['ntxh'].std()
dm['mvhhs'] = (dm['mvhh'] - dm['mvhh'].mean()) / dm['mvhh'].std()
dm['godhs'] = (dm['godinh'] - dm['godinh'].mean()) / dm['godinh'].std()

dm['cinths'] = (dm['cinth'] - dm['cinth'].mean()) / dm['cinth'].std()
dm['matus'] = (dm['matu'] - dm['matu'].mean()) / dm['matu'].std()

dm.head(2)

,id,session,sequence,seq,trips,nota,falk,rsos,tsos,grip,ptiso,pt60,godin,paq,ntxc,cint,calc,vitd,vitd2,mvh,matu,age,gender,height,seatedheight,leglength,weight,bf_slaughter,bia_pbf,bmi,lbmkg,yearsfromaphv,age_at_phv,bone_age,tanner,menarche,age_first_period,period_frequency,medications,vitamins,fracture,radius_z,tibial_z,wrist_flexion_best,flength,forearm_lean_csa,forearm_us_csa,anterior_csa,anteriorradius,ant_rad_csa,...,arm_circumference,arm_lean_csa,tlength,thigh_circumference,thigh_lean_csa,bicepsbrachii,v68,quads,v70,biodex_id,visit_one,visit_two,dob,agedays,ageyears,pt240,griph,ptisoh,rsosh,tsosh,flengthh,tlengthh,mvhh,sedh,ntxh,godinh,paqh,paqprvnth,calch,cinth,vitdh,boy,girl,rsoss,tsoss,grips,ptisos,ntxs,mvhs,gods,cints,matus,rsoshs,tsoshs,griphs,ptisohs,ntxhs,mvhhs,godhs,cinths
0,100,1,1,1,4.00,nan,nan,3828.00,3601.00,nan,113.93,69.29,37.00,2.59,711.85,1572.00,1077.67,3.38,135.27,105.71,-1.67,11.75,0,152.00,75.20,76.80,44.20,22.81,21.90,19.13,34.12,-2.13,13.88,11.16,2.00,nan,,,0.00,0.00,0.00,0.80,-0.75,nan,nan,nan,nan,nan,nan,nan,...,24.00,30.77,35.50,41.70,93.97,26.49,5.51,27.25,5.83,B6M100,05-05-2010,05-12-2010,08-11-1998,4292.00,11.75,41.80,23.77,113.93,3828.00,3601.00,23.77,35.50,105.71,78.43,711.85,37.00,2.59,1.00,1077.67,1572.00,3.38,1.00,nan,0.12,-0.79,nan,-0.22,-1.10,-0.01,-0.90,-0.05,-0.40,0.11,-0.81,0.06,-0.25,0.68,0.06,-0.90,-0.05
1,100,1,3,2,4.00,nan,nan,3898.00,3629.00,27.00,136.02,87.78,48.00,3.17,760.09,2219.01,1212.82,1.02,41.02,93.62,-0.71,12.71,0,160.80,79.60,81.20,48.80,17.24,14.90,18.87,40.39,-1.21,13.92,12.41,4.00,nan,,,0.00,0.00,0.00,1.00,-0.70,13.65,25.60,30.41,14.49,11.97,34.74,9.48,...,24.20,36.14,36.50,42.30,106.18,23.42,4.31,28.27,6.28,B6M200,04-20-2011,04-27-2011,08-11-1998,4642.00,12.71,52.60,27.00,136.02,3898.00,3629.00,25.60,36.50,93.62,75.55,760.09,48.00,3.17,1.00,1212.82,2219.01,1.02,1.00,nan,0.82,-0.53,0.53,0.19,-0.55,-0.30,-0.62,1.29,0.09,0.86,-0.55,0.54,0.22,0.87,-0.27,-0.62,1.29


In [80]:
%%stata --data dm 
set processors 20

save ./out/dm, replace


    The maximum number of processors or cores being used is changed from 32 to(note: file ./out/dm.dta not found)
file ./out/dm.dta saved


# Table 1. Summary statatisitcs

In [81]:
ds = dm.groupby('seq')['rsosh', 'tsosh', 'griph', 'ptisoh'].mean()
d0 = dm[dm['gender']==0].groupby('seq')['rsosh', 'tsosh', 'griph', 'ptisoh'].mean()
d1 = dm[dm['gender']==1].groupby('seq')['rsosh', 'tsosh', 'griph', 'ptisoh'].mean()

t1 = ds.append(d0)
t1 = t1.append(d1)
t1.to_csv("./out/t1.csv")
t1

,rsosh,tsosh,griph,ptisoh
seq,,,,
1,3790.33,3655.38,21.94,114.36
2,3825.27,3689.77,22.84,127.47
3,3850.35,3722.46,25.64,142.55
4,3884.24,3800.12,30.41,144.63
1,3789.40,3653.24,22.27,118.40
2,3813.99,3680.17,24.09,129.80
3,3835.70,3705.18,27.41,150.16
4,3850.76,3788.75,34.44,142.73
1,3791.44,3657.93,21.54,109.51


In [82]:
ds = dm.groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()
d0 = dm[dm['gender']==0].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()
d1 = dm[dm['gender']==1].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()

t1 = ds.append(d0)
t1 = t1.append(d1)
#t1.to_csv("./out/t1.csv")
t1

,rsos,tsos,grip,ptiso
seq,,,,
1,3792.21,3654.82,21.05,111.62
2,3820.22,3687.64,22.86,127.48
3,3842.10,3729.59,26.49,146.71
4,3880.53,3800.12,31.24,nan
1,3791.63,3652.71,21.62,116.70
2,3809.36,3676.25,24.28,131.64
3,3822.49,3705.89,28.82,157.85
4,3842.88,3788.75,36.19,nan
1,3792.89,3657.30,20.38,106.24


In [83]:
ds.iloc[3]/ds.iloc[0]-1
d0.iloc[3]/d0.iloc[0]-1
d1.iloc[3]/d1.iloc[0]-1

rsos    0.02
tsos    0.04
grip    0.48
ptiso    nan
dtype: float64

rsos    0.01
tsos    0.04
grip    0.67
ptiso    nan
dtype: float64

rsos    0.03
tsos    0.04
grip    0.32
ptiso    nan
dtype: float64

# Figure 2. Impact of muscle strength on bone properties

In [84]:
%%stata --data dm 
set processors 20

xtset id seq
gsem ///
    (M1[id] griphs matus ntxhs godin  -> rsoshs, ) /// 
    (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    (cinths matus  -> griphs,)  ///
    (cinths matus  -> torqhs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      cov(M1[id]*M2[id])  
        
estimates save "./out/table2_b"

lincom [griphs]matus - [torqhs]matus
lincom [griphs]cinths - [torqhs]cinths

lincom [rsoshs]griphs - [tsoshs]torqhs
lincom [rsoshs]matus - [tsoshs]matus
lincom [rsoshs]ntxhs - [tsoshs]ntxhs
lincom [rsoshs]paqps - [tsoshs]mvhhs





    The maximum number of processors or cores being used is changed from 32 to       panel variable:  id (unbalanced)
        time variable:  seq, 1 to 4
                delta:  1 unit
>     (M1[id] griphs matus ntxhs godin  -> rsoshs, ) /// 
>     (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
>     (cinths matus  -> griphs,)  ///
>     (cinths matus  -> torqhs,)  ///
>      , covstruct(_lexogenous, diagonal) nocapslatent ///
>       latent(M1 M2)   ///
>       cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
>       cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
>       cov(M1[id]*M2[id])  
variable torqhs not found;
Perhaps you meant 'torqhs' to specify a latent variable.
For 'torqhs' to be a valid latent variable specification, 'torqhs' must appear in the latent() option.
r(111);

end of do-file

r(111);


# Figure 3. Constrained specification results.

In [14]:
%%stata --data dm 
set processors 20

estimates use "./out/table2_b"
matrix b = e(b)
matrix list b

***********************************************************************

constraint 1 _b[rsoshs:griphs] = _b[tsoshs:torqhs]
constraint 2 _b[rsoshs:matus]  = _b[tsoshs:matus]
constraint 3 _b[rsoshs:ntxhs]  = _b[tsoshs:ntxhs]
constraint 4 _b[rsoshs:paqps]  = _b[tsoshs:mvhhs]

constraint 5 _b[griphs:cinths] = _b[torqhs:cinths]
constraint 6 _b[griphs:matus]  = _b[torqhs:matus]


constraint 10 _b[griphs:_cons] = 0 
constraint 11 _b[torqhs:_cons] = 0 
constraint 12 _b[rsoshs:_cons] = 0 
constraint 13 _b[tsoshs:_cons] = 0 

constraint 14 [var(e.rsoshs)]_cons - [var(e.tsoshs)]_cons = 0
constraint 15 [var(e.griphs)]_cons - [var(e.torqhs)]_cons = 0

gsem ///
    (M1[id]@1 griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M1[id]@1 torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    (  cinths matus    -> griphs,)  ///
    (  cinths matus    -> torqhs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      constraints(1 2 3 4 5 6  10 11 12 13 14 15) ///
      from(b, skip)

** indirect effect      
nlcom _b[rsoshs:c.griphs]*_b[griphs:c.matus]
** total effect      
nlcom _b[rsoshs:c.griphs] + _b[rsoshs:c.griphs]*_b[griphs:c.matus]


    The maximum number of processors or cores being used is changed from 32 to
b[1,29]
          rsoshs:       rsoshs:       rsoshs:       rsoshs:       rsoshs:       rsoshs:       tsoshs:       tsoshs:       tsoshs:       tsoshs:       tsoshs:       tsoshs:       griphs:       griphs:
          griphs         matus         ntxhs         paqps        M1[id]         _cons        torqhs         matus         ntxhs         mvhhs        M2[id]         _cons        cinths         matus
y1     .23692391     .31107759    -.17873268     .05521809             1    -.01837405     .39644358     .32353942    -.14388529     .08226188             1     -.0264097     .17173356     .56342181

          griphs:       torqhs:       torqhs:       torqhs:  var(M1[id]):  var(M2[id]):  cov(M2[id~):  var(e.rso~):  var(e.tso~):  var(e.gri~):  var(e.tor~):  cov(e.tso~):  cov(e.gri~):  cov(e.tor~):
           _cons        cinths         matus         _cons         _cons         _cons         _cons         _con

# Table 2. Sex-specific estimation results.

In [80]:
%%stata --data dm 
set processors 20

label define sex 0 "boys" 1 "girls
label values gender sex  



constraint 10 _b[griphs:_cons] = 0 
constraint 11 _b[torqhs:_cons] = 0 
constraint 12 _b[rsoshs:_cons] = 0 
constraint 13 _b[tsoshs:_cons] = 0 


constraint 20 [var(e.rsoshs)]_cons - [var(e.tsoshs)]_cons = 0
constraint 21 [var(e.griphs)]_cons - [var(e.torqhs)]_cons = 0
/*
gsem ///
    (M1[id]@1 i1.gender#c.griphs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
    (M1[id]@1 i1.gender#c.torqhs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.mvhhs  -> tsoshs, ) /// 
    ///
    (M1[id]@1 i0.gender#c.griphs i0.gender#c.matus i0.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
    (M1[id]@1 i0.gender#c.torqhs i0.gender#c.matus i0.gender#c.ntxhs i0.gender#c.mvhhs  -> tsoshs, ) /// 
    ///
    (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> griphs,)  ///
    (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> torqhs,)  ///
    ///
    (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> griphs,)  ///
    (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> torqhs,)  ///
    , covstruct(_lexogenous, diagonal) nocapslatent ///
    latent(M1) constraints(10 11 12 13 20 21) ///
    cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
    cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs)
     
estimates store t3a
estimates save "./out/t3", replace

lincom _b[rsoshs:0b.gender#c.griphs] - _b[tsoshs:0b.gender#c.torqhs]
lincom _b[rsoshs:i1.gender#c.griphs] - _b[tsoshs:i1.gender#c.torqhs]
lincom _b[rsoshs:0b.gender#c.matus] - _b[tsoshs:0b.gender#c.matus]
lincom _b[rsoshs:1.gender#c.matus] - _b[tsoshs:1.gender#c.matus]
lincom _b[rsoshs:0b.gender#c.ntxhs] - _b[tsoshs:0b.gender#c.ntxhs]
lincom _b[rsoshs:1.gender#c.ntxhs] - _b[tsoshs:1.gender#c.ntxhs]   
lincom _b[griphs:0b.gender#c.matus] - _b[torqhs:0b.gender#c.matus]    
lincom _b[griphs:1.gender#c.matus] - _b[torqhs:1.gender#c.matus]
*/
***********************************************************************
***********************************************************************
gsem ///
    (M1[id]@1 i1.gender#c.griphs@b1a i1.gender#c.matus@b1b i1.gender#c.ntxhs@b1c i1.gender#c.paqps@b1d  -> rsoshs, ) /// 
    (M1[id]@1 i1.gender#c.torqhs@b1a i1.gender#c.matus@b1b i1.gender#c.ntxhs@b1c i1.gender#c.mvhhs@b1d  -> tsoshs, ) /// 
    ///
    (M1[id]@1 i0.gender#c.griphs@b0a i0.gender#c.matus@b0b i0.gender#c.ntxhs@b0c i0.gender#c.paqps@b0d  -> rsoshs, ) /// 
    (M1[id]@1 i0.gender#c.torqhs@b0a i0.gender#c.matus@b0b i0.gender#c.ntxhs@b0c i0.gender#c.mvhhs@b0d  -> tsoshs, ) /// 
    ///
    (i1.gender#c.godhs@m1a i1.gender#c.cinths@m1b i1.gender#c.matus@m1c    -> griphs,)  ///
    (i1.gender#c.godhs@m1a i1.gender#c.cinths@m1b i1.gender#c.matus@m1c    -> torqhs,)  ///
    ///
    (i0.gender#c.godhs@m0a i0.gender#c.cinths@m0b i0.gender#c.matus@m0c  -> griphs,)  ///
    (i0.gender#c.godhs@m0a i0.gender#c.cinths@m0b i0.gender#c.matus@m0c  -> torqhs,)  ///
    , covstruct(_lexogenous, diagonal) nocapslatent ///
    latent(M1) constraints(10 11 12 13 20 21) ///
    cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
    cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs)

     estimates store t3b
estimates save "./out/t3b", replace

***********************************************************************
***muscle strength

lincom _b[griphs:1.gender#c.matus] - _b[griphs:0.gender#c.matus]

***************************
***nutrition
  
lincom _b[griphs:1.gender#c.cinths] - _b[griphs:0.gender#c.cinths]       
          
***************************
  
lincom _b[griphs:1.gender#c.godhs] - _b[griphs:0.gender#c.godhs] 
***************************
***bone properties
         
lincom _b[rsoshs:1.gender#c.matus] - _b[rsoshs:0.gender#c.matus]
***************************
    
lincom _b[rsoshs:1.gender#c.ntxhs] - _b[rsoshs:0.gender#c.ntxhs]
***************************
 
lincom _b[rsoshs:1.gender#c.griphs] - _b[rsoshs:0.gender#c.griphs]
          
***************************

lincom _b[tsoshs:1.gender#c.mvhhs] - _b[tsoshs:0.gender#c.mvhhs]          
          
***********************************************************************
****decompose effects maturation effect
          
****boys
**direct effect
 
*matu on bone          
nlcom _b[rsoshs:0b.gender#c.matus]    
         
** indirect effect 
*grip on bone
nlcom _b[rsoshs:0b.gender#c.griphs]

*matu on grip
nlcom _b[griphs:0b.gender#c.matus]

*product (indirect effect)
nlcom _b[rsoshs:0b.gender#c.griphs]*_b[griphs:0b.gender#c.matus]
         
** total effect      
    
nlcom _b[rsoshs:0b.gender#c.matus] 
nlcom _b[rsoshs:0b.gender#c.griphs]*_b[griphs:0b.gender#c.matus]
nlcom _b[rsoshs:0b.gender#c.matus] + _b[rsoshs:0b.gender#c.griphs]*_b[griphs:0b.gender#c.matus]

***girls
**direct effect
 
*matu on bone          
nlcom _b[rsoshs:i1.gender#c.matus]    
         
** indirect effect 
*grip on bone
nlcom _b[rsoshs:i1.gender#c.griphs]

*matu on grip
nlcom _b[griphs:i1.gender#c.matus]

*product (indirect effect)
nlcom _b[rsoshs:i1.gender#c.griphs]*_b[griphs:i1.gender#c.matus]
         
** total effect      
    
nlcom _b[rsoshs:i1.gender#c.matus] 
nlcom _b[rsoshs:i1.gender#c.griphs]*_b[griphs:i1.gender#c.matus]
nlcom _b[rsoshs:i1.gender#c.matus] + _b[rsoshs:i1.gender#c.griphs]*_b[griphs:i1.gender#c.matus]
         
************************************************************************************
************************************************************************************
************************************************************************************
 
*estout t3a t3b , cells("b( fmt(2)) ci(par fmt(2))") interaction(:) label 
         


    The maximum number of processors or cores being used is changed from 32 to
. > gsem ///
>     (M1[id]@1 i1.gender#c.griphs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
>     (M1[id]@1 i1.gender#c.torqhs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.mvhhs  -> tsoshs, ) /// 
>     ///
>     (M1[id]@1 i0.gender#c.griphs i0.gender#c.matus i0.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
>     (M1[id]@1 i0.gender#c.torqhs i0.gender#c.matus i0.gender#c.ntxhs i0.gender#c.mvhhs  -> tsoshs, ) /// 
>     ///
>     (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> griphs,)  ///
>     (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> torqhs,)  ///
>     ///
>     (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> griphs,)  ///
>     (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> torqhs,)  ///
>     , covstruct(_lexogenous, diagonal) nocapslatent ///
>     latent(M1) constraints(10 11 12 13 20 21) ///
>     cov(e

# Table 3a. Male-specific results.

In [50]:
%%stata --data dm 
set processors 20

constraint 1 _b[rsoshs:griphs] = _b[tsoshs:torqhs]
constraint 2 _b[rsoshs:matus]  = _b[tsoshs:matus]
constraint 3 _b[rsoshs:ntxhs]  = _b[tsoshs:ntxhs]
constraint 4 _b[rsoshs:paqps]  = _b[tsoshs:mvhhs]

constraint 5 _b[griphs:godhs]  = _b[torqhs:godhs]
constraint 6 _b[griphs:cinths] = _b[torqhs:cinths]
constraint 7 _b[griphs:matus]  = _b[torqhs:matus]


constraint 10 _b[griphs:_cons] = 0 
constraint 11 _b[torqhs:_cons] = 0 
constraint 12 _b[rsoshs:_cons] = 0 
constraint 13 _b[tsoshs:_cons] = 0 

constraint 14 [var(e.rsoshs)]_cons - [var(e.tsoshs)]_cons = 0
constraint 15 [var(e.griphs)]_cons - [var(e.torqhs)]_cons = 0

gsem ///
    (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    (godhs cinths matus    -> griphs,)  ///
    (godhs cinths matus    -> torqhs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      cov(M1[id]*M2[id])  
matrix b = e(b)

gsem ///
    (M1[id]@1 griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M1[id]@1 torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    ( godhs cinths matus    -> griphs,)  ///
    ( godhs cinths matus    -> torqhs,)  ///
     if gender==0, covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      constraints(1 2 3 4 5 6 7  10 11 12 13 14 15) ///
      from(b, skip)





    The maximum number of processors or cores being used is changed from 32 to>     (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
>     (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
>     (godhs cinths matus    -> griphs,)  ///
>     (godhs cinths matus    -> torqhs,)  ///
>      , covstruct(_lexogenous, diagonal) nocapslatent ///
>       latent(M1 M2)   ///
>       cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
>       cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
>       cov(M1[id]*M2[id])  

Fitting fixed-effects model:

Iteration 0:   log likelihood = -2050.3787  (not concave)
Iteration 1:   log likelihood =  -1986.265  
Iteration 2:   log likelihood = -1982.0747  
Iteration 3:   log likelihood = -1971.8136  
Iteration 4:   log likelihood = -1970.9114  
Iteration 5:   log likelihood =  -1970.826  
Iteration 6:   log likelihood = -1970.8253  
Iteration 7:   log likelihood = -1970.8253  

Refining starting values:

Grid node 0:   log likelihood = -1947.9747

Fi

# Table 3b. Girl-specific results.

In [48]:
%%stata --data dm 
set processors 20

constraint 1 _b[rsoshs:griphs] = _b[tsoshs:torqhs]
constraint 2 _b[rsoshs:matus]  = _b[tsoshs:matus]
constraint 3 _b[rsoshs:ntxhs]  = _b[tsoshs:ntxhs]
constraint 4 _b[rsoshs:paqps]  = _b[tsoshs:mvhhs]

constraint 5 _b[griphs:godhs]  = _b[torqhs:godhs]
constraint 6 _b[griphs:cinths] = _b[torqhs:cinths]
constraint 7 _b[griphs:matus]  = _b[torqhs:matus]

constraint 10 _b[griphs:_cons] = 0 
constraint 11 _b[torqhs:_cons] = 0 
constraint 12 _b[rsoshs:_cons] = 0 
constraint 13 _b[tsoshs:_cons] = 0 

constraint 14 [var(e.rsoshs)]_cons - [var(e.tsoshs)]_cons = 0
constraint 15 [var(e.griphs)]_cons - [var(e.torqhs)]_cons = 0

gsem ///
    (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    (godhs cinths matus    -> griphs,)  ///
    (godhs cinths matus    -> torqhs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      cov(M1[id]*M2[id])  
matrix b = e(b)

gsem ///
    (M1[id]@1 griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M1[id]@1 torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    ( godhs cinths matus    -> griphs,)  ///
    ( godhs cinths matus    -> torqhs,)  ///
     if gender==1, covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      constraints(1 2 3 4 5 6 7  10 11 12 13 14 15) ///
      from(b, skip)

** indirect effect      
nlcom _b[rsoshs:c.griphs]*_b[griphs:c.matus]
** total effect      
nlcom _b[rsoshs:c.griphs] + _b[rsoshs:c.griphs]*_b[griphs:c.matus]


    The maximum number of processors or cores being used is changed from 32 to>     (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
>     (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
>     (godhs cinths matus    -> griphs,)  ///
>     (godhs cinths matus    -> torqhs,)  ///
>      , covstruct(_lexogenous, diagonal) nocapslatent ///
>       latent(M1 M2)   ///
>       cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
>       cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
>       cov(M1[id]*M2[id])  

Fitting fixed-effects model:

Iteration 0:   log likelihood = -2050.3787  (not concave)
Iteration 1:   log likelihood =  -1986.265  
Iteration 2:   log likelihood = -1982.0747  
Iteration 3:   log likelihood = -1971.8136  
Iteration 4:   log likelihood = -1970.9114  
Iteration 5:   log likelihood =  -1970.826  
Iteration 6:   log likelihood = -1970.8253  
Iteration 7:   log likelihood = -1970.8253  

Refining starting values:

Grid node 0:   log likelihood = -1947.9747

Fi